# Simulating the world cup

In [ ]:
import itertools
import operator
import numpy as np
import pandas as pd
import pystan

In [ ]:
group_a = ("Japan", "Ireland", "Scotland", "Samoa", "Russia")
group_b = ("New Zealand", "Italy", "Namibia", "Canada", "South Africa")
group_c = ("France", "England", "United States of America", "Tonga", "Argentina")
group_d = ("Australia", "Wales", "Georgia", "Uruguay", "Fiji")

all_teams = group_a + group_b + group_c + group_d

In [ ]:
df = pd.read_csv("rugby_data.csv")

df = df[
    df["home_team"].isin(all_teams) & df["away_team"].isin(all_teams) & (df["date"] < "2019-09-19")
]

team_indices = {team: i + 1 for i, team in enumerate(all_teams)}
df = df.replace(
    to_replace={"home_team": team_indices, "away_team": team_indices}
)

## Stan model

In [ ]:
stan_code = """
data {
    int<lower=1> nteam;
    int<lower=1> nmatch;
    int home_team[nmatch];
    int away_team[nmatch];
    int home_points[nmatch];
    int away_points[nmatch];
}
parameters {
    vector[nteam] log_a_tilde;
    vector[nteam] log_b_tilde;
    real<lower=0> sigma_a;
    real<lower=0> sigma_b;
    real mu_b;
    real<lower=0> phi;
}
transformed parameters {
    vector[nteam] a = exp(sigma_a * log_a_tilde);
    vector[nteam] b = exp(mu_b + sigma_b * log_b_tilde);
    vector[nmatch] home_rate = a[home_team] .* b[away_team];
    vector[nmatch] away_rate = a[away_team] .* b[home_team];
}
model {
    phi ~ normal(0, 5);
    sigma_a ~ normal(0, 1);
    sigma_b ~ normal(0, 1);
    mu_b ~ normal(0, 5);
    log_a_tilde ~ normal(0, 1);
    log_b_tilde ~ normal(0, 1);
    home_points ~ neg_binomial_2(home_rate, phi);
    away_points ~ neg_binomial_2(away_rate, phi);
}
generated quantities {
    int home_points_rep[nmatch];
    int away_points_rep[nmatch];
    for (i in 1:nmatch) {
        home_points_rep[i] = neg_binomial_2_rng(home_rate[i], phi);
        away_points_rep[i] = neg_binomial_2_rng(away_rate[i], phi);
    }
}
"""

model = pystan.StanModel(model_code=stan_code)

stan_data = {
    "nteam": len(all_teams),
    "nmatch": len(df),
    "home_team": df["home_team"].values,
    "away_team": df["away_team"].values,
    "home_points": df["home_points"].values,
    "away_points": df["away_points"].values
}

fit = model.sampling(stan_data, seed=42)

## Simulate the cup

In [ ]:
a = fit["a"]
b = fit["b"]
phi = fit["phi"]

attack = {team: a[:, i] for i, team in enumerate(all_teams)}
defense = {team: b[:, i] for i, team in enumerate(all_teams)}

attack_list = [{team: attack[team][i] for team in all_teams} for i in range(a.shape[0])]
defense_list = [{team: defense[team][i] for team in all_teams} for i in range(b.shape[0])]

In [ ]:
def convert_params(mu, phi):
    var = mu + 1 / phi * mu ** 2
    p = (var - mu) / var
    return phi, 1 - p

def simulate_group_match(a_home, a_away, b_home, b_away, phi):
    home_score = np.random.negative_binomial(*convert_params(a_home * b_away, phi))
    away_score = np.random.negative_binomial(*convert_params(a_away * b_home, phi))
    
    home_points = 0
    away_points = 0
    
    if home_score > away_score:
        home_points += 4
        if home_score - away_score <= 7:
            away_points += 1
    elif home_score < away_score:
        away_points += 4
        if away_score - home_score <= 7:
            home_points += 1
    else:
        home_points += 2
        away_points += 2
        
    if home_score > 25:
        home_points += 1
    
    if away_score > 25:
        away_points += 1
        
    return home_points, away_points

def simulate_knockout_match(home_team, away_team, attack, defense, phi):
    a_home, a_away = attack[home_team], attack[away_team]
    b_home, b_away = defense[home_team], defense[away_team]
    home_score = np.random.negative_binomial(*convert_params(a_home * b_away, phi))
    away_score = np.random.negative_binomial(*convert_params(a_away * b_home, phi))
    if home_score == away_score:
        return simulate_knockout_match(home_team, away_team, attack, defense, phi)
    elif home_score > away_score:
        return home_team
    else:
        return away_team
    

def simulate_group(teams, attack, defense, phi):
    group_points = {team: 0 for team in teams}
    for home_team, away_team in itertools.combinations(teams, 2):
        home_points, away_points = simulate_group_match(
            attack[home_team], attack[away_team], defense[home_team], defense[away_team], phi
        )
        group_points[home_team] += home_points
        group_points[away_team] += away_points
        
    top_two = [
        team for team, points in reversed(sorted(group_points.items(), key=lambda item: item[1]))
    ][:2]
    
    return top_two[0], top_two[1]


def simulate_world_cup(attack, defense, phi):
    
    # simulate the group matches
    a_winner, a_runner_up = simulate_group(group_a, attack, defense, phi)
    b_winner, b_runner_up = simulate_group(group_b, attack, defense, phi)
    c_winner, c_runner_up = simulate_group(group_c, attack, defense, phi)
    d_winner, d_runner_up = simulate_group(group_d, attack, defense, phi)
    
    # quarter finals
    qf1_winner = simulate_knockout_match(c_winner, d_runner_up, attack, defense, phi)
    qf2_winner = simulate_knockout_match(b_winner, a_runner_up, attack, defense, phi)
    qf3_winner = simulate_knockout_match(d_winner, c_runner_up, attack, defense, phi)
    qf4_winner = simulate_knockout_match(a_winner, b_runner_up, attack, defense, phi)
    
    # semi finals
    sf1_winner = simulate_knockout_match(qf1_winner, qf2_winner, attack, defense, phi)
    sf2_winner = simulate_knockout_match(qf3_winner, qf4_winner, attack, defense, phi)
    
    # the final
    cup_winner = simulate_knockout_match(sf1_winner, sf2_winner, attack, defense, phi)
    
    return {
        "a_winner": a_winner,
        "a_runner_up": a_runner_up,
        "b_winner": b_winner,
        "b_runner_up": b_runner_up,
        "c_winner": c_winner,
        "c_runner_up": c_runner_up,
        "d_winner": d_winner,
        "d_runner_up": d_runner_up,
        "qf1_winner": qf1_winner,
        "qf2_winner": qf2_winner,
        "qf3_winner": qf3_winner,
        "qf4_winner": qf4_winner,
        "sf1_winner": sf1_winner,
        "sf2_winner": sf2_winner,
        "cup_winner": cup_winner
    }

## Who will win?

In [ ]:
np.random.seed(42)
results = pd.DataFrame([simulate_world_cup(a, b, p) for a, b, p in zip(attack_list, defense_list, phi)])
df_results = pd.DataFrame(results["cup_winner"].value_counts() / len(results)).reset_index()

df_results.columns = ["Team", "Probability of winning the world cup"]

missing_teams = set(all_teams) - set(df_results["Team"])
df_missing = pd.DataFrame({
    "Team": list(missing_teams),
    "Probability of winning the world cup": [0.0,] * len(missing_teams)
})

df_results = pd.concat((df_results, df_missing))
df_results.to_csv("results.csv", index=False)

## Model checking

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(4, 4, figsize=(10, 10))
for i, axi in enumerate(ax.ravel()):
    sim_label = "simulated data"
    data_label = "true data"
    sns.distplot(fit["home_points_rep"][i, :], kde=False, ax=axi, label=sim_label)
    sns.distplot(
        df["home_points"], 
        kde=False, 
        hist_kws={"histtype": "step", "lw": 2, "color": "k"}, 
        ax=axi, 
        label=data_label
    )
    axi.set_xlabel("")
    if i == 0:
        axi.legend()
     

    fig.tight_layout()
plt.savefig("points_distro.png")

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(10, 10))
for i, axi in enumerate(ax.ravel()):
    sim_label = "simulated data"
    data_label = "true data"
    sns.distplot(
        fit["home_points_rep"][i, :] - fit["away_points_rep"][i, :], 
        hist_kws={"histtype": "step", "lw": 2, "color": "r"},
        kde=False,
        ax=axi, 
        label=sim_label
    )
    sns.distplot(
        df["home_points"] - df["away_points"], 
        kde=False, 
        hist_kws={"histtype": "step", "lw": 2, "color": "k"}, 
        ax=axi, 
        label=data_label
    )
    if i == 0:
        axi.legend()
        
fig.tight_layout()
plt.savefig("difference_distro.png")

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(10, 10))
for i, axi in enumerate(ax.ravel()):
    sim_label = "simulated data"
    data_label = "true data"
    sns.distplot(
        fit["home_points_rep"][i, :] + fit["away_points_rep"][i, :], 
        hist_kws={"histtype": "step", "lw": 2, "color": "r"},
        kde=False,
        ax=axi, 
        label=sim_label
    )
    sns.distplot(
        df["home_points"] + df["away_points"], 
        kde=False, 
        hist_kws={"histtype": "step", "lw": 2, "color": "k"}, 
        ax=axi, 
        label=data_label
    )
    if i == 0:
        axi.legend()
        
fig.tight_layout()
plt.savefig("total_points_distro.png")

## England's fate

In [ ]:
pr_out_of_group = ((results["c_winner"] == "England") | (results["c_runner_up"] == "England")).sum() / len(results)

In [ ]:
pr_semi_final = ((results["qf1_winner"] == "England") | (results["qf3_winner"] == "England")).sum() / len(results)

In [ ]:
pr_final = ((results["sf1_winner"] == "England") | (results["sf2_winner"] == "England")).sum() / len(results)

In [ ]:
print(pr_out_of_group)
print(pr_semi_final)
print(pr_final)

## Group summaries

In [ ]:
for group in ["a", "b", "c", "d"]:
    df_group_win = pd.DataFrame(results[f"{group}_winner"].value_counts() / len(results)).reset_index()
    df_group_win.columns = ["Team", f"Probability of winning group {group}"]
    df_group_win.to_csv(f"{group}_win.csv", index=False)
    
    df_group_runner = pd.DataFrame(results[f"{group}_runner_up"].value_counts() / len(results)).reset_index()
    df_group_runner.columns = ["Team", f"Probability of being runner up of {group}"]
    df_group_runner.to_csv(f"{group}_runner.csv", index=False)

## Knockout redux

In [ ]:
def simulate_world_cup_knockouts(attack, defense, phi):
    
    # quarter finals
    qf1_winner = simulate_knockout_match("England", "Australia", attack, defense, phi)
    qf2_winner = simulate_knockout_match("New Zealand", "Ireland", attack, defense, phi)
    qf3_winner = simulate_knockout_match("Wales", "France", attack, defense, phi)
    qf4_winner = simulate_knockout_match("Japan", "South Africa", attack, defense, phi)
    
    # semi finals
    sf1_winner = simulate_knockout_match(qf1_winner, qf2_winner, attack, defense, phi)
    sf2_winner = simulate_knockout_match(qf3_winner, qf4_winner, attack, defense, phi)
    
    # the final
    cup_winner = simulate_knockout_match(sf1_winner, sf2_winner, attack, defense, phi)
    
    return {
        "qf1_winner": qf1_winner,
        "qf2_winner": qf2_winner,
        "qf3_winner": qf3_winner,
        "qf4_winner": qf4_winner,
        "sf1_winner": sf1_winner,
        "sf2_winner": sf2_winner,
        "cup_winner": cup_winner
    }

In [ ]:
np.random.seed(43)
knockout_results = pd.DataFrame(
    [simulate_world_cup_knockouts(a, b, p) for a, b, p in zip(attack_list, defense_list, phi)]
)
df_knockout_results = pd.DataFrame(
    knockout_results["cup_winner"].value_counts() / len(knockout_results)
).reset_index()

df_knockout_results.columns = ["Team", "Probability of winning the world cup"]

df_knockout_results.round(decimals=3).to_csv("knockout_results.csv", index=False)